In [1]:
import pyspark

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('test') \
    .getOrCreate()

25/03/05 09:33:55 WARN Utils: Your hostname, MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 100.112.89.122 instead (on interface en0)
25/03/05 09:33:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/05 09:33:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# GroupBy

In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.registerTempTable('green')

/opt/anaconda3/lib/python3.11/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [13]:
df_green_revenue = spark.sql("""
SELECT 
  date_trunc('hour', lpep_pickup_datetime) AS hour
, PULocationID AS zone 
, SUM(total_amount) AS amount
, COUNT(1) AS number_records
FROM green 
WHERE 1=1
      AND lpep_pickup_datetime >= '2020-01-01'
GROUP BY 1,2
""")

In [14]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-11 14:00:00|  42|247.86000000000004|            19|
|2020-01-07 20:00:00|  92|              80.6|             5|
|2020-01-28 16:00:00| 223|            241.33|            15|
|2020-01-12 21:00:00| 181| 325.9000000000001|            19|
|2020-01-27 18:00:00| 165|             32.11|             1|
|2020-01-07 17:00:00| 126|            131.31|             4|
|2020-01-04 10:00:00|  69|232.78000000000003|             7|
|2020-01-29 16:00:00| 133|             47.46|             2|
|2020-01-19 10:00:00| 205| 64.25999999999999|             3|
|2020-01-08 14:00:00| 140|            212.39|             6|
|2020-01-18 22:00:00|  49|             24.68|             2|
|2020-01-31 22:00:00| 181|            648.09|            30|
|2020-01-25 18:00:00|  95|229.65000000000006|            18|
|2020-01-10 17:00:00|  4

In [15]:
df_green_revenue. \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode = 'overwrite')

25/03/05 09:41:01 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/03/05 09:41:01 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/03/05 09:41:01 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/03/05 09:41:01 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/03/05 09:41:01 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/03/05 09:41:01 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/03/05 09:41:01 WARN MemoryManager: Total allocation exceeds 95.0

In [16]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [17]:
df_yellow_revenue = spark.sql("""
SELECT 
  date_trunc('hour', tpep_pickup_datetime) AS hour
, PULocationID AS zone 
, SUM(total_amount) AS amount
, COUNT(1) AS number_records
FROM yellow 
WHERE 1=1
      AND tpep_pickup_datetime >= '2020-01-01'
GROUP BY 1,2
""")

In [19]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode = 'overwrite')

25/03/05 09:52:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/03/05 09:52:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/03/05 09:52:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/03/05 09:52:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/03/05 09:52:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/03/05 09:52:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/03/05 09:52:33 WARN MemoryManager: Total allocation exceeds 95.0

# Join

Let's materialize our results for more efficient processing (without this next bit of code, we'd be recomputing everything each time we run!)

In [30]:
## USING PREVIOUS RESULTS!
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [31]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records') 

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records') 

In [32]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on = ['hour', 'zone'], how = 'outer')

In [34]:
df_join.write.parquet('data/report/revenue/total', mode = 'overwrite')

25/03/05 10:12:11 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/03/05 10:12:11 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/03/05 10:12:11 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/03/05 10:12:11 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/03/05 10:12:12 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/03/05 10:12:12 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/03/05 10:12:12 WARN MemoryManager: Total allocation exceeds 95.0

In [28]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  36|295.34000000000003|                  11|            109.17|                    3|
|2020-01-01 00:00:00|  45|              NULL|                NULL| 732.4800000000002|                   42|
|2020-01-01 00:00:00|  59|50.900000000000006|                   3|              NULL|                 NULL|
|2020-01-01 00:00:00|  76|            143.78|                   4|             35.51|                    2|
|2020-01-01 00:00:00|  79|              NULL|                NULL|12573.810000000034|                  721|
|2020-01-01 00:00:00|  90|              NULL|                NULL| 5010.450000000003|                  266|
|2020-01-01 00:00:00|  92|  

## Broadcast Join

In [37]:
df_zones = spark.read.parquet('data/zones/*')

In [38]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [39]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [43]:
# df_result.drop('LocationID').show()
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue_zones', mode = 'overwrite')

25/03/05 10:21:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/03/05 10:21:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/03/05 10:21:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/03/05 10:21:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/03/05 10:21:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/03/05 10:21:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/03/05 10:21:44 WARN MemoryManager: Total allocation exceeds 95.0

In [44]:
# spark.stop()